In [2]:
import logging
import time
import os
import numpy
from scipy import sparse
from pandas import DataFrame
from scipy.sparse import coo_matrix
from collections import defaultdict, Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import implicit
import h5py
import pandas as pd
import numpy as np
import pickle
from itertools import chain
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import pandas as pd

In [7]:
#from evaluation import rr, get_user_precision

In [1]:
data_path="/Users/mismael/Documents/projects/ContextualRecsMXNetDev/data/orders-5c-train.csv"

In [3]:
df=pd.read_csv(data_path)


In [8]:
df.head()

,bb938286-487e-11e4-9697-9cb654858910,433593,2018-05-10 21:13:08.000,late night
0,bc443afe-487e-11e4-9697-9cb654858910,676517,2018-05-09 17:00:44.000,dinner
1,bcbee2c2-487e-11e4-9697-9cb654858910,308694,2018-05-29 20:55:09.000,dinner
2,bd3c036a-487e-11e4-9697-9cb654858910,303952,2018-05-16 15:05:51.000,mid-day
3,bfabb938-487e-11e4-9697-9cb654858910,383154,2018-05-23 19:23:57.000,dinner
4,bfdcb966-487e-11e4-9697-9cb654858910,80559,2018-05-12 16:10:48.000,dinner


In [9]:

data_path_test="/Users/mismael/Documents/projects/ContextualRecsMXNetDev/data/orders-5c-test.csv"


In [9]:
#read train and test

def create_user_logs(data_path):
    user_item_dict=defaultdict(list)
    with open (data_path) as data_file:
        for line in data_file:
            line_list = line.split(",")

            user=line_list[0]
            #print (user)
            rest=line_list[1]
            user_item_dict[user].append(rest)
    #print (len (user_item_dict))
    return user_item_dict



def create_user_test_logs(data_path_test):
    user_item_dict_test=defaultdict(list)
    with open (data_path_test) as data_file:
        for line in data_file:
            line_list = line.split(",")

            user=line_list[0]
            print (user)
            rest=line_list[1]
            user_item_dict_test[user].append(rest)
    #print (len (user_item_dict_test))
    return user_item_dict_test


In [11]:
user_item_dict=create_user_logs(data_path)
print (len (user_item_dict))

#user_item_dict_test=create_user_test_logs(data_path_test)
#print (len (user_item_dict_test))


707765


In [12]:
# find frequency of items per user

def find_rest_freq(user_item_dict):
    user_list_freq= []
    for user in user_item_dict:
        #print (user)
        list_of_rests = user_item_dict[user]
        rest_counter = Counter(list_of_rests)
        #print (rest_counter)
        for res_id in rest_counter:

            freq = rest_counter[res_id]
            user_list_freq.append([user, res_id, freq])
            #print (user_list_freq)
            #print (len (user_list_freq))

    headers = ['user', 'restaurant', 'frequency']
    datafram = DataFrame(user_list_freq, columns=headers)

    #print (user_list_freq)
    return datafram


#user_item_dict=create_user_logs(data_path)

In [13]:
datafram=find_rest_freq(user_item_dict)


<module 'posixpath' from '/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/posixpath.py'>

In [14]:
#datafram['user']=datafram['user'].index.values.astype(int)

In [15]:
datafram.to_csv("dinner_item_freq.csv")

In [16]:
#Build Spars matrix or build prefrence item user matrix. 

def build_ui_matrix(datafram):
    datafram['user']=datafram['user'].astype("category")
    datafram['restaurant']=datafram['restaurant'].astype("category")
    user_order_log_matrix= coo_matrix((datafram['frequency'].astype(numpy.float32),
                       (datafram['restaurant'].cat.codes.copy(),
                        datafram['user'].cat.codes.copy())))
    
    return user_order_log_matrix

In [17]:
item_user_matrix=build_ui_matrix(datafram)

In [18]:
# make a modeel

model = implicit.nearest_neighbours.BM25Recommender()
model.fit(item_user_matrix)

100%|██████████| 5826/5826 [00:00<00:00, 7679.33it/s]


In [19]:
user_items = item_user_matrix.T.tocsr()

In [20]:
from itertools import islice
def reco ( model,userid, user_items, N=10 ):
        """ returns the best N recommendations for a user given its id"""
        # recalculate_user is ignored because this is not a model based algorithm
        items = N     
        indices, data = model.scorer.recommend(userid, user_items.indptr, user_items.indices,                                              user_items.data, K=items, remove_own_likes=False)
        best = sorted(zip(indices, data), key=lambda x: -x[1]) 
        return list(islice((rec for rec in best ), N))
        #return list(itertools.islice((rec for rec in best if rec[0] not in liked), N))

In [21]:
#recomend item to training dataset and create output

rests = dict(enumerate(datafram['restaurant'].cat.categories))
with open('output_bm25_train', 'w') as out:
    for userid, username in enumerate(datafram['user'].cat.categories.tolist()):
        for rest_index, score in reco(model,userid, user_items):            
            out.write("" + str(username) + ","+ str(rests[rest_index])+ "," +str (score)+ "%\n")           

In [22]:
train_predict='/Users/mismael/output_bm25_train'

In [23]:
# make disition for each user in training dateset

def create_user_item_predict(train_predict):
    user_item_predict=defaultdict(list)
    with open (train_predict) as data_file:
        for line in data_file:
            line_list = line.split(",")

            user=line_list[0]
            rest=line_list[1]
            user_item_predict[user].append(rest)
    print (len(user_item_predict))
    return user_item_predict

In [24]:
user_item_precict_result=create_user_item_predict(train_predict)


707765


In [25]:
# test Bm25 with predicted train with acutal test

user_test=0
k=4
num_past_orders=[]
verbose=False
precision=[]
recall=[]
i=0
won = 0
for key in user_item_dict_test:    

    if key in user_item_precict_result:

        predictions=user_item_precict_result[key]
        
        targets=user_item_dict_test[key]
        #hithit = len(set(predictions).intersection(set(actual_lastweek)))
    
        #print ('target',type(targets[0]))
                               
        predictions = predictions[:k] # get top k goto restaurants
        #print ("p", type(predictions[0]))
        #print (len (predictions))
        num_hit = len(set(predictions).intersection(set(targets))) #get true positives

        user_precision = float(num_hit) / len(predictions) #divide by k
        #print (user_precision)
        user_recall = float(num_hit) / len(targets) 
        #print (user_recall)
        precision.append(user_precision)
        recall.append(user_recall)
        

print(i, "p: ", np.mean(np.array(precision)), "r: ", np.mean(np.array(recall)) )
        
              

0 p:  0.12911487194847984 r:  0.47653163192290043


In [27]:
# test Bm25 with predicted train with acutal train


user_test=0
k=15
num_past_orders=[]
verbose=False
precision=[]
recall=[]
i=0
won = 0
for key in user_item_dict:    

    if key in user_item_precict_result:

        predictions=user_item_precict_result[key]
        
        targets=user_item_dict[key]
        #hithit = len(set(predictions).intersection(set(actual_lastweek)))
    
        #print ('target',type(targets[0]))
                               
        predictions = predictions[:k] # get top k goto restaurants
        #print ("p", type(predictions[0]))
        #print (len (predictions))
        num_hit = len(set(predictions).intersection(set(targets))) #get true positives

        user_precision = float(num_hit) / len(predictions) #divide by k
        #print (user_precision)
        user_recall = float(num_hit) / len(targets) 
        #print (user_recall)
        precision.append(user_precision)
        recall.append(user_recall)
        

print(i, "p: ", np.mean(np.array(precision)), "r: ", np.mean(np.array(recall)) )
        
 

0 p:  0.5830443720726515 r:  0.32940496344098


In [28]:
user_test=0
k=15
num_past_orders=[]
verbose=False
precision=[]
recall=[]
i=0
for key in user_item_dict_test.keys():    
    #print (key)
    if key in user_item_precict_result.keys():
        #print (key )
        predictions=user_item_precict_result[key]
        
        targets=user_item_dict_test[key]
        #print (len (targets))
                               
        predictions = predictions[:k] # get top k goto restaurants
        #print (predictions)
        num_hit = len(set(predictions).intersection(set(targets))) #get true positives
        #print (num_hit)
        user_precision = float(num_hit) / len(predictions) #divide by k
        #print (user_precision)
        user_recall = float(num_hit) / len(targets) 
        #print (user_recall)
        precision.append(user_precision)
        recall.append(user_recall)
print(i, "p: ", np.array(precision).squeeze().mean(), "r: ", np.array(recall).squeeze().mean())

0 p:  0.06477609704957316 r:  0.5936376246687763


In [29]:
#user_item_dict_test_goto=create_user_test_logs_goto(data_path_test_goto)
#print (len (user_item_dict_test_goto))

# ALS

In [30]:
model1 = implicit.als.AlternatingLeastSquares(factors=50)



In [31]:
model1.fit(item_user_matrix)

100%|██████████| 15.0/15 [00:32<00:00,  2.11s/it]


In [32]:
#user_items = item_user_matrix.T.tocsr()

In [33]:
type(model1.user_factors.shape), model1.item_factors.shape

(tuple, (5826, 50))

In [34]:
recommendations = model1.recommend(userid, user_items)

In [35]:

related = model.similar_items(3)

In [36]:
user_index_id={}
for userid, username in enumerate(datafram['user'].cat.categories.tolist()):   
    user_index_id[userid]=(username)


In [37]:
item_latent=defaultdict(list)
for item_inex, headen_item in enumerate (model1.item_factors):   
    item_latent[item_inex].append(headen_item)

In [38]:
len(item_latent)

5826

In [39]:
def recol ( model,userid, user_items, N=10 ):
        """ returns the best N recommendations for a user given its id"""
        # recalculate_user is ignored because this is not a model based algorithm
        items = N     
        indices, data = model1.recommend(userid, user_items.indptr, user_items.indices,                                              user_items.data, K=items, remove_own_likes=False)
        best = sorted(zip(indices, data), key=lambda x: -x[1]) 
        return list(islice((rec for rec in best ), N))
rests = dict(enumerate(datafram['restaurant'].cat.categories))
with open('output_ALS', 'w') as out:
    for userid, username in enumerate(datafram['user'].cat.categories.tolist()):
        for rest_index, score in model1.recommend(userid, user_items):        
            out.write("" + str(username) + ","+ str(rests[rest_index])+ "," +str (score)+ "%\n")      

In [40]:
train_predict_ALS='output_ALS'

In [41]:
def create_user_item_predict_ALS(train_predict_ALS):
    user_item_predict_ALS=defaultdict(list)
    with open (train_predict_ALS) as data_file:
        for line in data_file:
            line_list = line.split(",")

            user=line_list[0]
            rest=line_list[1]
            user_item_predict_ALS[user].append(rest)
    print (len (user_item_predict_ALS))
    return user_item_predict_ALS

user_item_precict_result_ALS=create_user_item_predict_ALS(train_predict_ALS)
print (len (user_item_precict_result_ALS))

707765
707765


In [42]:
user_test=0
k=4
num_past_orders=[]
verbose=False
precision_ALS=[]
recall_ALS=[]
i=0
for key in user_item_dict_test.keys():    
    #print (key)
    if key in user_item_precict_result_ALS.keys():
        #print (key )
        predictions=user_item_precict_result_ALS[key]
        
        targets=user_item_dict_test[key]
        #print (len (targets))
                               
        predictions = predictions[:k] # get top k goto restaurants
        #print (len (predictions))
        num_hit = len(set(predictions).intersection(set(targets))) #get true positives
        #print (num_hit)
        user_precision = float(num_hit) / len(predictions) #divide by k
        #print (user_precision)
        user_recall = float(num_hit) / len(targets) 
        #print (user_recall)
        precision_ALS.append(user_precision)
        recall_ALS.append(user_recall)
print(i, "p : ALS", np.array(precision_ALS).mean(), "r: ", np.array(recall_ALS).mean())
        
               

0 p : ALS 0.00296914782087764 r:  0.010946782487144925


In [43]:
#dot product between useritem.itemuser=u.u
        
def get_related( user_factors, item_factors, N=50):
        # scores = self.factors.dot(self.factors[artistid])
        scores = item_factors.dot(user_factors)
        best = numpy.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])
    

In [45]:
user_index_id={}
for userid, username in enumerate(datafram['user'].cat.categories.tolist()):       
    user_index_id[userid]=(username)    

In [46]:
user_index_id={}
for userid, username in enumerate(datafram['user'].cat.categories.tolist()):   
    user_index_id[userid]=(username)


user_latent_index=defaultdict(list)
for user_inex, headen_item in enumerate (model1.user_factors): 
    #print (user_inex)
    real_user=user_index_id[user_inex]
    #print (real_user)
    user_latent_index[real_user].append(headen_item)
    
    
with open('user_latent_index.pickle', 'wb') as handle:
    pickle.dump(user_latent_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('user_latent_index.pickle', 'rb') as handle:
    b = pickle.load(handle)  

In [47]:
item_index_id={}
for userid, username in enumerate(datafram['restaurant'].cat.categories.tolist()):   
    item_index_id[userid]=(username)
    
item_latent_index=defaultdict(list)
for user_inex, headen_item in enumerate (model1.item_factors): 
    real_item=item_index_id[user_inex]
    #print (real_user)
    item_latent_index[real_item].append(headen_item)    
    

with open('item_latent_index.pickle', 'wb') as handle:
    pickle.dump(item_latent_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('item_latent_index.pickle', 'rb') as handle:
    h = pickle.load(handle)  

In [48]:
item_factors = model1.item_factors

recommended = defaultdict(list)
for index, user_factor in enumerate(model1.user_factors):

    for item_index, score in get_related(user_factor,item_factors, N=10):
        #print(item_index)
        item_value=item_index_id[item_index]
        
        recommended[index].append((item_value,score))  

In [49]:
# preaper data to evaluate
user_item_latent = defaultdict(list)
for user_index  in recommended:
    real_user=user_index_id[user_index]
    score=recommended[user_index]
    item_=([i[0] for i in score])
    user_item_latent[real_user].append(item_)

In [50]:
#Evaluate ALS (latent features ) predicted training data set with test

user_test=0
k=4
num_past_orders=[]
precision_ALS_latent=[]
recall_ALS_latent=[]
i=0
for key in user_item_dict_test.keys():    
    #print (key)
    if key in user_item_latent.keys():

        predictions=user_item_latent[key]
        prediction=list(chain.from_iterable(predictions))
        targets=user_item_dict_test[key]        
        #print (targets)                               
        prediction = prediction[:k] # get top k goto restaurants
        #print (prediction)
        #print (prediction)
        #print (targets)
        #print (len (predictions))
        num_hit = len(set(prediction).intersection(set(targets))) #get true positives
        #print (num_hit)
        user_precision = float(num_hit) / len(prediction) #divide by k
        #print (user_precision)
        if len (targets)>0:
            user_recall = float(num_hit) / len(targets) 
            #print (user_recall)
        precision_ALS_latent.append(user_precision)
        recall_ALS_latent.append(user_recall)
print(i, "p : ALS", np.array(precision_ALS_latent).mean(), "r: ", np.array(recall_ALS_latent).mean())
                  

0 p : ALS 0.05755578852778194 r:  0.21349183765163998


In [51]:
#Evaluate ALs (latent features )with predicted traininga data set with actual dataset

user_test=0
k=4
num_past_orders=[]

precision_ALS_latent=[]
recall_ALS_latent=[]
i=0
for key in user_item_dict:    
    #print (key)
    if key in user_item_latent:

        predictions=user_item_latent[key]
        prediction=list(chain.from_iterable(predictions))
        #print(prediction)
        
        targets=user_item_dict[key]        
        #print (targets)
                               
        prediction = prediction[:k] # get top k goto restaurants
        #print (prediction)
        #print (prediction)
        #print (targets)
        #print (len (predictions))
        num_hit = len(set(prediction).intersection(set(targets))) #get true positives
        #print (num_hit)
        user_precision = float(num_hit) / len(prediction) #divide by k
        #print (user_precision)
        if len (targets)>0:
            user_recall = float(num_hit) / len(targets) 
            #print (user_recall)
        precision_ALS_latent.append(user_precision)
        recall_ALS_latent.append(user_recall)
print(i, "p : ALS", np.array(precision_ALS_latent).squeeze().mean(), "r: ", np.array(recall_ALS_latent).squeeze().mean())
                  




0 p : ALS 0.39454727204651263 r:  0.08598053370833918


# Normilize by Bm25 and ALS

In [52]:
logging.debug("weighting matrix by bm25_weight")
normal_user_item_matrix = bm25_weight(item_user_matrix)
als_normal_bm= implicit.als.AlternatingLeastSquares()
als_normal_bm.fit(normal_user_item_matrix)

100%|██████████| 15.0/15 [00:43<00:00,  3.10s/it]


In [53]:
als_normal_bm.user_factors.shape, als_normal_bm.item_factors.shape

((707765, 100), (5826, 100))

In [54]:
#h=normal_user_item_matrix.todense()

In [56]:
item_latent_bm25=defaultdict(list)
for item_inex, headen_item in enumerate (als_normal_bm.item_factors):   
    item_latent_bm[item_inex].append(headen_item)

NameError: name 'item_latent_bm' is not defined

In [58]:
def get_related_als_normlize( item_factors_als_normal,user_factor,  N=50):
        # scores = self.factors.dot(self.factors[artistid])
    
        scores = item_factors_als_normal.dot(user_factor)
        best = numpy.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])
    
      
item_factors_als_normal = als_normal_bm.item_factors
recommended_als = defaultdict(list)
for index, user_factor in enumerate(als_normal_bm.user_factors):


    for item_index, score in get_related_als_normlize(item_factors_als_normal,user_factor, N=10):
        item_value=item_index_id[item_index]
        recommended_als[index].append((item_value,score))     

In [59]:
user_item_latent_norm = defaultdict(list)
for user_index  in recommended_als:
    #print (user_index)
    real_user=user_index_id[user_index]
    score=recommended_als[user_index]
    item_=([i[0] for i in score])
    user_item_latent_norm[real_user].append(item_)

In [60]:
user_test=0
k=10
num_past_orders=[]

precision_ALS_latent_norm=[]
recall_ALS_latent_norm=[]
i=0
for key in user_item_dict_test.keys():    
    #print (key)
    if key in user_item_latent_norm.keys():

        predictions=user_item_latent_norm[key]
        prediction=list(chain.from_iterable(predictions))   
        targets=user_item_dict_test[key]        
                               
        prediction = prediction[:k] # get top k goto restaurants

        num_hit = len(set(prediction).intersection(set(targets))) #get true positives
        #print (num_hit)
        user_precision = float(num_hit) / len(prediction) #divide by k
        #print (user_precision)
        if len (targets)>0:
            user_recall = float(num_hit) / len(targets) 
            #print (user_recall)
        precision_ALS_latent_norm.append(user_precision)
        recall_ALS_latent_norm.append(user_recall)
print(i, "mean.precision: ALS", np.array(precision_ALS_latent_norm).mean(), "mean.recall: ", np.array(recall_ALS_latent_norm).mean())
                  

0 mean.precision: ALS 0.04277519844241426 mean.recall:  0.39657822690846506


# ALS normolize BM25

In [61]:
logging.debug("weighting matrix by bm25_weight")
normal_user_item_matrixT = bm25_weight(item_user_matrix.T)
als_normal_bmT= implicit.als.AlternatingLeastSquares(factors=50)
als_normal_bmT.fit(normal_user_item_matrixT)

100%|██████████| 15.0/15 [00:30<00:00,  1.90s/it]


In [62]:
print (len(als_normal_bmT.user_factors))
print (len (als_normal_bmT.item_factors))


5826
707765


In [63]:
def get_related_als_normlizeT( user_factors_als_normalT,item_factor,  N=50):
        # scores = self.factors.dot(self.factors[artistid])
        scores = user_factors_als_normalT.dot(item_factor)
        best = numpy.argpartition(scores, -N)[-N:]
        return sorted(zip(best, scores[best]), key=lambda x: -x[1])
    
      
item_factors_als_normalT = als_normal_bmT.item_factors
user_factors_als_normalT = als_normal_bmT.user_factors
recommended_alsT = defaultdict(list)
for index, item_factor in enumerate(item_factors_als_normalT):


    for item_index, score in get_related_als_normlizeT(user_factors_als_normalT,item_factor, N=10):
        item_value=item_index_id[item_index]
        recommended_alsT[index].append((item_value,score))  

In [64]:
user_item_latent_norm_T = defaultdict(list)
for user_index  in recommended_alsT:
    #print (user_index)
    real_user=user_index_id[user_index]
    score=recommended_alsT[user_index]
    item_=([i[0] for i in score])
    user_item_latent_norm_T[real_user].append(item_)

In [65]:
user_test=0
k=10
num_past_orders=[]

precision_ALS_latent_normT=[]
recall_ALS_latent_normT=[]
i=0
for key in user_item_dict_test.keys():    
    #print (key)
    if key in user_item_latent_norm_T.keys():

        predictions=user_item_latent_norm_T[key]
        prediction=list(chain.from_iterable(predictions))   
        targets=user_item_dict_test[key]        
                               
        prediction = prediction[:k] # get top k goto restaurants

        num_hit = len(set(prediction).intersection(set(targets))) #get true positives
        #print (num_hit)
        user_precision = float(num_hit) / len(prediction) #divide by k
        #print (user_precision)
        if len (targets)>0:
            user_recall = float(num_hit) / len(targets) 
            #print (user_recall)
        precision_ALS_latent_normT.append(user_precision)
        recall_ALS_latent_normT.append(user_recall)
print(i, "mean.precision: ALS", np.array(precision_ALS_latent_normT).mean(), "mean.recall: ", np.array(recall_ALS_latent_normT).mean())
                  

0 mean.precision: ALS 0.03862812640407368 mean.recall:  0.3563834330597099
